In [ ]:
# -*- coding: utf-8 -*-
%reload_ext autoreload
%autoreload 2
import sys
def defp(pt):
    try:    sys.path.index(pt)
    except: sys.path.append(pt)
defp('/ml.dev/bin')
defp('/ml.dev/lib/oanda/oandapy')
import matplotlib.pylab as plt
import seaborn as ses
ses.set()
%pylab inline
from qoreliquid import *
import oandapyV20
import oandapyV20.endpoints.accounts as accounts
import oandapyV20.endpoints.trades as trades
import oandapyV20.endpoints.positions as positions
from oandapyV20.endpoints.pricing import PricingStream
from oandapyV20.endpoints.accounts import AccountSummary
import pandas as p
import numpy as n
import oandapy
from matplotlib import pyplot as plt
from pylab import rcParams
%pylab inline
rcParams['figure.figsize'] = 20, 5
accountID = "101-004-1984564-001"
co = p.read_csv('/mldev/bin/datafeeds/config.csv', header=None)
loginIndex = 4
env0=co.ix[loginIndex,1]
access_token0=co.ix[loginIndex,2]
print 'access_token0 %s' % access_token0
oanda0 = oandapy.API(environment=env0, access_token=access_token0)
#client = API(access_token=access_token, headers={"Content-Type": "application/json"})
client = oandapyV20.API(access_token=access_token0)
#client = oandapyV20.API(access_token=access_token0, environment="live")
#client = oandapyV20.API(access_token=access_token0, environment="practice")
#client = oandapyV20.API(access_token=access_token0)
#client.api_url = 'https://test.com'
print pf(co)
#for i in co[2]: print i
ac = p.read_csv('/mldev/bin/datafeeds/accounts.csv', header=0)
print pf(ac)
accountID = ac.ix[1, 'id']

In [ ]:
# percentage loss gain table
import numpy as n
import pandas as p
# r = [0:100]'
# a = [r (100-r)/100*100-100 100./(100-r)*100-100]
r = n.arange(110, dtype=float)
df = p.DataFrame(r, columns=['pcnt'])
df['lossPcnt']     = (100 - r) / 100 * 100 - 100
df['gainBackPcnt'] = 100 / (100 - r) * 100 - 100
df['diffPcnt'] = n.abs(df['gainBackPcnt']) - n.abs(df['lossPcnt'])
with p.option_context('display.max_rows', 4000, 'display.max_columns', 4000, 'display.width', 1000000):
    print df
df.to_csv('/tmp/lossGainTable.csv')

In [ ]:
import numpy as n
df = p.DataFrame(n.arange(1,12*10))
#df[1] = 219.80 * n.power(1+1.0/100, df[0])
df[2] = 1000.0 * n.power(1+5.0/100, df[0])
%pylab inline
rcParams['figure.figsize'] = 25, 5
df.plot()

In [ ]:
r = trans.TransactionsSinceID(accountID=accountID, params={'id':4000})
client.request(r)
print r.response

In [ ]:
def cacheFile(url, fname):
    import os, time, requests
    stat = os.stat(fname)
    if time.time() - stat.st_mtime >= 30:
        # read/download
        r   = requests.get(url)
        res = r.text
        
        fp = open(fname, 'w')
        fp.write(res)
        fp.close()
        
        return res
    else:
        # cache
        fp = open(fname, 'r')
        res = fp.read()
        fp.close()
        print res


cacheFile('http://google.com', '/tmp/qweqwe')

In [ ]:
api?

In [ ]:
import oandapyV20
import oandapyV20.endpoints.transactions as trans
def cacheOandapyV20Request(r, fname, age=30000):
    import os, time, requests
    import ujson as j
    
    def makeRequest(r):
        # read/download
        print 'requesting..'
        co = p.read_csv('/mldev/bin/datafeeds/config.csv', header=None)
        loginIndex = 2
        env0=co.ix[loginIndex,1]
        access_token0=co.ix[loginIndex,2]
        client = oandapyV20.API(access_token=access_token0)
        #print client.request(r)
        rv = client.request(r)
        # write to cache file
        fp = open(fname, 'w')
        fp.write(j.dumps(rv))
        fp.close()
        return rv
    
    def cacheRequest(r):
        print 'caching..'
        # cache
        fp = open(fname, 'r')
        rv = fp.read()
        rv = j.loads(rv)
        fp.close()
        return rv

    try:
        stat = os.stat(fname)
        if time.time() - stat.st_mtime >= age:
            return makeRequest(r)
        else:
            return cacheRequest(r)
    except:
        return makeRequest(r)

rv = cacheOandapyV20Request(accounts.AccountList(), '/tmp/qweqwe'+"-AccountList.json")
df = p.DataFrame(rv['accounts'])
#print df
accountIndex = 2
accountID = "101-004-1984564-006"
accountID = df.ix[accountIndex,'id']
print '%s %s' % (env0, access_token0)
print 'accountID:%s ' % accountID
#r = cacheRequest(accounts.AccountList(), '/tmp/qweqwe'+"-AccountList.json")
r = trans.TransactionList(accountID)  # params optional
client.request(r)
import drest
with p.option_context('display.max_rows', 4000, 'display.max_columns', 4000, 'display.width', 1000000):
    df = p.DataFrame(r.response)
    print 'accountID: %s' % accountID
    print df
    print 
    params = {
        #"to": df.ix[0,'lastTransactionID'],
        "to": df.ix[0,'lastTransactionID'],
        "from": 1
    }
    r = trans.TransactionIDRange(accountID=accountID, params=params)
    client.request(r)
    print r.response
    for i in df.ix[:, 'pages']:#[0:2]:
        print i
        #api = drest.API(i)
        #api?
        #print api.make_request('GET', i)
        #trans.


In [ ]:
import oandapyV20
import oandapyV20.endpoints.transactions as trans
#client = oandapyV20.API(access_token=...)
accountID = "101-004-1984564-006"
def st():
    params = {
              "to": 400,
              "from": 1
    }
    r = trans.TransactionIDRange(accountID=accountID, params=params)
    client.request(r)
    #print p.DataFrame(r.response)
    df = p.DataFrame()
    with p.option_context('display.max_rows', 4000, 'display.max_columns', 4000, 'display.width', 1000000):
        for i in r.response['transactions']:
            dfi = p.DataFrame(i, index=[0])
            dfi['id'] = p.to_numeric(dfi['id'])
            df = df.combine_first(dfi.set_index('id'))
    df['accountBalance'] = p.to_numeric(df['accountBalance'])
    df['accountBalance'] = df['accountBalance'].bfill().ffill()
    #print df.dtypes
    #print df#.sort_index()
    df = df.ix[:,'accountBalance']
    #print df 
    %pylab inline
    df.plot()
"""
for i in r.response['transactions']:
    print i
"""

In [ ]:
# compound interest vector
df = 100 * n.power(1+1.0/100, range(0,300)) - 100
plt.plot(df)
plt.legend('test', loc=2)

In [ ]:
# bank statement simulator
import time
df = p.DataFrame([])
ra = range(1,12*2)
for i in ra:
    df.ix[time.time()+(60*60*24*30*i),'deposit'] = 6500.0*20/100/15
df = df.sort_index()
df = df.fillna(0)
for i in range(len(df.index)):
    if i > 0:    df.ix[df.index[i], 'interest'] = df.ix[df.index[i-1],'balance'] * n.power(1+12.0/100, 1) - df.ix[df.index[i-1],'balance'] #+ df.ix[df.index[i],'deposit']
    #balance * pow(1+1.0/100, seconds)
    elif i == 0: df.ix[df.index[i], 'interest'] = 0
    if i > 0:    df.ix[df.index[i], 'balance'] = df.ix[df.index[i-1],'balance'] + df.ix[df.index[i],'deposit'] + df.ix[df.index[i],'interest']
    elif i == 0: df.ix[df.index[i], 'balance'] = df.ix[df.index[i],'deposit']
df['balance'] = df['balance'].ffill()
df['month'] = ra
plot(df.index, df['balance'])
print df.ix[:,'month deposit interest balance'.split(' ')]

In [ ]:
# metatrader portfolio prototype
df = p.read_csv('/mldev/bin/data/oanda/cache/patterns/patterns.portfolioMetatrader.1484941956.48.csv', index_col=[0])
with p.option_context('display.max_rows', 4000, 'display.max_columns', 4000, 'display.width', 1000000):
    print df

In [ ]:
# compute metatrader portfolio
#from qoreliquid import computeMetraderData
from qoreliquid import Patterns
df = p.read_csv('/mldev/bin/data/oanda/cache/patterns/patterns.portfolioMetatrader.1485150677.45.csv', index_col=[0])
pa = Patterns()
df = pa.computePortfolioMetatrader(df, balance=140.65, leverage=500)
import numpy as n
df['lots'] = n.round(df['lots'], 2)
with p.option_context('display.max_rows', 4000, 'display.max_columns', 4000, 'display.width', 1000000):
    print df.ix[:, 'side lots balanceMetatrader leverageMetatrader allMarginMetatrader amount2Metatrader closeTradePLMetatrader side diffp lots'.split(' ')].sort_index()

In [ ]:
#0.68/n.pi
print n.log(2)/n.log(1+float(1)/100)
r = ((n.log(2)/n.log(1+float(1)/100))/100)
print r
print 72.0/r
print 69.3/r

In [ ]:
100*n.power(1+2.0/100,35.0027887811)

In [ ]:
# altcoin history
#http://drest.readthedocs.io/en/0.9.6/quickstart.html
#http://coincap.io
#https://github.com/CoinCapDev/CoinCap.io
import drest
#api = drest.API('http://socket.coincap.io/')
api = drest.API('http://www.coincap.io/')

def getCoincapIoHistory(coinCode):
    #/history/*Timeframe*/<coinCode>
    # GET http://localhost:8000/api/v1/users/1/
    response = api.make_request('GET', '/history/1day/%s'%coinCode)
    #print response.data
    with p.option_context('display.max_rows', 4000, 'display.max_columns', 4000, 'display.width', 1000000):
        df = p.DataFrame(response.data)
        #print df.ix[0,:]
        df['ts']          = map(lambda x: x[0], df['price'])
        df['price0_%s'%coinCode]      = map(lambda x: x[1], df['price'])
        df['market_cap0_%s'%coinCode] = map(lambda x: x[1], df['market_cap'])
        df['volume0_%s'%coinCode]     = map(lambda x: x[1], df['volume'])
        df = df.set_index('ts')
        #print df
        df = df.ix[:, ('price0_%s'%coinCode).split()]#.plot()
        #print df
        return df

df = p.DataFrame([])

res = api.make_request('GET', '/coins')
coins = p.DataFrame(res.data).set_index(0)
for i in coins.index[0:100]:
    #print i
    try:
        df = df.combine_first(getCoincapIoHistory(i))
    except: ''
#df = df.combine_first(getCoincapIoHistory('BTC'))
#df = df.combine_first(getCoincapIoHistory('ETH'))
pdf = df.bfill().ffill()
#print pdf
from qoreliquid import normalizeme
df = normalizeme(pdf)
from pylab import rcParams
%pylab inline
rcParams['figure.figsize'] = 20, 5
df.plot()

from pylab import rcParams
rcParams['figure.figsize'] = 20, 5
%pylab inline
#df.plot()
plt.plot(df)
plt.show()

In [ ]:
fname = '/opt/data/coincap.io.history.1day.csv'
df.to_csv(fname)
#df = p.read_csv(fname)
#df

In [ ]:
#http://coincap.io
response = api.make_request('GET', '/coins')
#print response.data
with p.option_context('display.max_rows', 4000, 'display.max_columns', 4000, 'display.width', 1000000):
    print p.DataFrame(response.data)

In [ ]:
#http://coincap.io
resources = [
    'coins', # Array of all supported coins
    'coins/xcp', # Array of all supported XCP assets
    'coins/xcp/all', # Array all XCP assets known
    'map', # JSON object of Coin symbols/names and known aliases
    'front', # Front Page Data of current coin stats
    'front/xcp', # Front Page Data of just XCP assets
    'global', # Global data
]
for i in resources:
    try:
        #print i
        api.add_resource(i)
    except Exception as e:
        #print e
        ''
api.resources

In [ ]:
res = api.coins.get()
print p.DataFrame(res.data)

In [ ]:
res = api.map.get()
print p.DataFrame(res.data)

In [ ]:
with p.option_context('display.max_rows', 4000, 'display.max_columns', 4000, 'display.width', 1000000):
    print p.DataFrame(res.data)

In [ ]:
#http://coincap.io
res = api.front.get()
#with p.option_context('display.max_rows', 4000, 'display.max_columns', 4000, 'display.width', 1000000):
df = p.DataFrame(res.data).set_index('short')
#df['volume'] = p.to_numeric(df['volume'])
#df = df.sort_values(by='volume')
df#.ix[:,'price']

In [ ]:
res = api.global.get()
print p.DataFrame(res.data)

In [ ]:
import numpy as n
df = p.DataFrame(n.arange(1,1000))
df[1] = 219.80 * n.power(1+1.0/100, df[0])
%pylab inline
df.plot()

In [ ]:
r = trans.TransactionsSinceID(accountID=accountID, params={'id':4000})
client.request(r)
print r.response

In [ ]:
import oandapyV20
import oandapyV20.endpoints.transactions as trans
r = trans.TransactionList(accountID)  # params optional
client.request(r)
with p.option_context('display.max_rows', 4000, 'display.max_columns', 4000, 'display.width', 1000000):
    df = p.DataFrame(r.response)
    print 'accountID: %s' % accountID
    print df
    for i in df.ix[:, 'pages']:
        print i
        trans.

In [ ]:
import oandapyV20
import oandapyV20.endpoints.transactions as trans
#client = oandapyV20.API(access_token=...)
accountID = "101-004-1984564-006"
def st():
    params = {
              "to": 400,
              "from": 1
    }
    r = trans.TransactionIDRange(accountID=accountID, params=params)
    client.request(r)
    #print p.DataFrame(r.response)
    df = p.DataFrame()
    with p.option_context('display.max_rows', 4000, 'display.max_columns', 4000, 'display.width', 1000000):
        for i in r.response['transactions']:
            dfi = p.DataFrame(i, index=[0])
            dfi['id'] = p.to_numeric(dfi['id'])
            df = df.combine_first(dfi.set_index('id'))
    df['accountBalance'] = p.to_numeric(df['accountBalance'])
    df['accountBalance'] = df['accountBalance'].bfill().ffill()
    #print df.dtypes
    #print df#.sort_index()
    df = df.ix[:,'accountBalance']
    #print df 
    %pylab inline
    df.plot()
"""
for i in r.response['transactions']:
    print i
"""

In [ ]:
# compound interest vector
100 * n.power(1+1.0/100, range(0,100)) - 100

In [ ]:
# bank statement simulator
import time
df = p.DataFrame([])
ra = range(1,12*2)
for i in ra:
    df.ix[time.time()+(60*60*24*30*i),'deposit'] = 6500.0*20/100/15
df = df.sort_index()
df = df.fillna(0)
for i in range(len(df.index)):
    if i > 0:    df.ix[df.index[i], 'interest'] = df.ix[df.index[i-1],'balance'] * n.power(1+12.0/100, 1) - df.ix[df.index[i-1],'balance'] #+ df.ix[df.index[i],'deposit']
    #balance * pow(1+1.0/100, seconds)
    elif i == 0: df.ix[df.index[i], 'interest'] = 0
    if i > 0:    df.ix[df.index[i], 'balance'] = df.ix[df.index[i-1],'balance'] + df.ix[df.index[i],'deposit'] + df.ix[df.index[i],'interest']
    elif i == 0: df.ix[df.index[i], 'balance'] = df.ix[df.index[i],'deposit']
df['balance'] = df['balance'].ffill()
df['month'] = ra
plot(df.index, df['balance'])
print df.ix[:,'month deposit interest balance'.split(' ')]

In [ ]:
# metatrader portfolio prototype
df = p.read_csv('/mldev/bin/data/oanda/cache/patterns/patterns.portfolioMetatrader.1484941956.48.csv', index_col=[0])
with p.option_context('display.max_rows', 4000, 'display.max_columns', 4000, 'display.width', 1000000):
    print df

In [ ]:
# compute metatrader portfolio
#from qoreliquid import computeMetraderData
from qoreliquid import Patterns
df = p.read_csv('/mldev/bin/data/oanda/cache/patterns/patterns.portfolioMetatrader.1485150677.45.csv', index_col=[0])
pa = Patterns()
df = pa.computePortfolioMetatrader(df, balance=140.65, leverage=500)
import numpy as n
df['lots'] = n.round(df['lots'], 2)
with p.option_context('display.max_rows', 4000, 'display.max_columns', 4000, 'display.width', 1000000):
    print df.ix[:, 'side lots balanceMetatrader leverageMetatrader allMarginMetatrader amount2Metatrader closeTradePLMetatrader side diffp lots'.split(' ')].sort_index()

In [ ]:
#0.68/n.pi
print n.log(2)/n.log(1+float(1)/100)
r = ((n.log(2)/n.log(1+float(1)/100))/100)
print r
print 72.0/r
print 69.3/r

In [ ]:
100*n.power(1+2.0/100,35.0027887811)

In [ ]:
# altcoin history
#http://drest.readthedocs.io/en/0.9.6/quickstart.html
#http://coincap.io
#https://github.com/CoinCapDev/CoinCap.io
import drest
#api = drest.API('http://socket.coincap.io/')
api = drest.API('http://www.coincap.io/')

def getCoincapIoHistory(coinCode):
    #/history/*Timeframe*/<coinCode>
    # GET http://localhost:8000/api/v1/users/1/
    response = api.make_request('GET', '/history/1day/%s'%coinCode)
    #print response.data
    with p.option_context('display.max_rows', 4000, 'display.max_columns', 4000, 'display.width', 1000000):
        df = p.DataFrame(response.data)
        #print df.ix[0,:]
        df['ts']          = map(lambda x: x[0], df['price'])
        df['price0_%s'%coinCode]      = map(lambda x: x[1], df['price'])
        df['market_cap0_%s'%coinCode] = map(lambda x: x[1], df['market_cap'])
        df['volume0_%s'%coinCode]     = map(lambda x: x[1], df['volume'])
        df = df.set_index('ts')
        #print df
        %pylab inline
        df = df.ix[:, ('price0_%s'%coinCode).split()]#.plot()
        #print df
        return df

df = p.DataFrame([])

res = api.make_request('GET', '/coins')
coins = p.DataFrame(res.data).set_index(0)
for i in coins.index[0:100]:
    #print i
    try:
        df = df.combine_first(getCoincapIoHistory(i))
    except: ''
#df = df.combine_first(getCoincapIoHistory('BTC'))
#df = df.combine_first(getCoincapIoHistory('ETH'))
df = df.bfill().ffill()
#print df
from qoreliquid import normalizeme
df = normalizeme(df)
from pylab import rcParams
%pylab inline
rcParams['figure.figsize'] = 20, 5
df.plot()

from pylab import rcParams
rcParams['figure.figsize'] = 20, 5
%pylab inline
#df.plot()
plt.plot(df)
plt.show()

In [ ]:
#http://coincap.io
response = api.make_request('GET', '/coins')
#print response.data
with p.option_context('display.max_rows', 4000, 'display.max_columns', 4000, 'display.width', 1000000):
    print p.DataFrame(response.data)

In [ ]:
#http://coincap.io
resources = [
    'coins', # Array of all supported coins
    'coins/xcp', # Array of all supported XCP assets
    'coins/xcp/all', # Array all XCP assets known
    'map', # JSON object of Coin symbols/names and known aliases
    'front', # Front Page Data of current coin stats
    'front/xcp', # Front Page Data of just XCP assets
    'global', # Global data
]
for i in resources:
    try:
        #print i
        api.add_resource(i)
    except Exception as e:
        #print e
        ''
api.resources

In [ ]:
res = api.coins.get()
print p.DataFrame(res.data)

In [ ]:
res = api.map.get()
print p.DataFrame(res.data)

In [ ]:
with p.option_context('display.max_rows', 4000, 'display.max_columns', 4000, 'display.width', 1000000):
    print p.DataFrame(res.data)

In [ ]:
#http://coincap.io
res = api.front.get()
#with p.option_context('display.max_rows', 4000, 'display.max_columns', 4000, 'display.width', 1000000):
df = p.DataFrame(res.data).set_index('short')
#df['volume'] = p.to_numeric(df['volume'])
#df = df.sort_values(by='volume')
df#.ix[:,'price']

In [ ]:
res = api.global.get()
print p.DataFrame(res.data)

In [ ]:
import oandapyV20
import oandapyV20.endpoints.orders as orders
data = {
    "order": {
        "stopLossOnFill": {
            "timeInForce": "GTC",
            "price": "1.05"
        },
        "type": "MARKET",
        "positionFill": "DEFAULT",
        "instrument": "EUR_USD",
        "timeInForce": "FOK",
        "units": "200"
    }
}
accountID = "101-004-1984564-006"
r = orders.OrderCreate(accountID, data=data)
client.request(r)
#print r.response
#p.DataFrame(r.response['orderCreateTransaction'])
res = r.response
#print p.DataFrame(r.response['orderFillTransaction'])

In [ ]:
#accid = oanda0.get_accounts()['accounts'][0]['accountId']
#instruments = p.DataFrame(oanda0.get_instruments(accid)['instruments']).set_index('instrument')

currentPositions = oanda0.get_positions(5545219)['positions']
currentPositions = p.DataFrame(currentPositions).set_index('instrument')
instr = ','.join(list(currentPositions.index))
print instr
currentPrices = oanda0.get_prices(instruments=instr)['prices']
#instruments
#currentPrices
currentPositions

In [ ]:
accid = '101-004-1984564-001'
r = accounts.AccountInstruments(accid)
rv = client.request(r)
df = p.DataFrame(rv['instruments']).set_index('name')
#with p.option_context('display.max_rows', 4000, 'display.max_columns', 4000, 'display.width', 1000000):
print 'instruments 00103'
#print p.DataFrame(df.columns)
#print df.ix[:,'displayName name maxTradeUnits pipLocation'.split()]
#print df
mdf = instruments.combine_first(df)
mdf

In [ ]:
import oandapyV20.endpoints.instruments as instruments
import oandapyV20.endpoints.transactions as transactions
import oandapyV20.endpoints.accounts as accounts

#transactions.TransactionList?
r = accounts.AccountInstruments('101-004-1984564-001')
rv = client.request(r)
with p.option_context('display.max_rows', 4000, 'display.max_columns', 4000, 'display.width', 1000000):
    print p.DataFrame(rv['instruments'])

al = accounts.AccountList()
print al
ral = client.request(al)
ral
df = p.DataFrame(ral['accounts'])
print df

In [ ]:
r = accounts.AccountList()
print r
#print r.ENDPOINT
print r.METHOD
print r.status_code
print r.response
#print client.request(r)
rv = client.request(r)
df = p.DataFrame(rv['accounts'])
print df

In [ ]:
r = accounts.AccountList()
rv = client.request(r)
df = p.DataFrame(rv['accounts'])
print df

In [ ]:
r = accounts.AccountList()
rv = client.request(r)
df = p.DataFrame(rv['accounts'])
print df
for i in df['id']:
    print 
    print '=========='
    print i
    r = trades.TradesList(i)
    rv = client.request(r)
    df = p.DataFrame(rv['trades'])
    with p.option_context('display.max_rows', 4000, 'display.max_columns', 4000, 'display.width', 1000000):
        try:
            #print df.dtypes
            #df = df.convert_objects(convert_numeric=True)
            df['unrealizedPL'] = p.to_numeric(df['unrealizedPL'])
            mdf = df.sort_values(by='unrealizedPL', ascending=False)
            #print mdf[mdf['unrealizedPL'] > 0]
            print mdf
        except: ''

In [ ]:
import json
from oandapyV20 import API
from oandapyV20.exceptions import V20Error
from oandapyV20.endpoints.pricing import PricingStream

instruments = "DE30_EUR,EUR_USD,EUR_JPY"
s = PricingStream(accountID=accountID, params={"instruments":instruments})
try:
    n = 0
    for R in client.request(s):
        with p.option_context('display.max_rows', 4000, 'display.max_columns', 4000, 'display.width', 1000000):
            print
            print '=========='
            print R['instrument']
            #print(json.dumps(R, indent=2))
            ptdf = p.DataFrame(R)
            print ptdf.ix[:,'closeoutAsk closeoutBid instrument status time tradeable type'.split(' ')]
            print '=== asks:'
            print p.DataFrame(R['asks'])
            print '=== bids:'
            print p.DataFrame(R['bids'])
        n += 1
        if n > 10:
            s.terminate("maxrecs received: {}".format(MAXREC))

except V20Error as e:
    print("Error: {}".format(e))
except Exception as e:
    print ''

In [ ]:
#trades.OpenTrades(accountID)
from oandapyV20.contrib import requests
from oandapyV20.endpoints import orders
mktOrder = requests.MarketOrderRequest(instrument="EUR_USD",
     units=100,
     takeProfitOnFill=requests.TakeProfitDetails(price=1.10).data,
     stopLossOnFill=requests.StopLossDetails(price=1.07).data
)
p.DataFrame(mktOrder.data)

# create the OrderCreate request
r = orders.OrderCreate(accountID, data=mktOrder.data)
try:
    # create the OrderCreate request
    rv = client.request(r)
except oandapyV20.exceptions.V20Error as err:
    print(r.status_code, err)
else:
    print p.DataFrame([rv]).transpose()
    import json
    print json.dumps(rv, indent=4)

In [ ]:
r = accounts.AccountList()
print r
#print r.ENDPOINT
print r.METHOD
print r.status_code
print r.response
#print client.request(r)
rv = client.request(r)
df = p.DataFrame(rv['accounts'])
print df

In [ ]:
from qoreliquid import Patterns
pa = Patterns()
pa.monitorAccountsMarginCloseout()

In [ ]:
pa.monitorAccountsProfitableTrades(verbose=True)

In [ ]:
accid = 5545219
r = oanda0.get_accounts()
print p.DataFrame(r['accounts'])
acco = oanda0.get_account(accid)
maccount = p.DataFrame(acco, index=[0])
print maccount.transpose()
acc = oanda0.get_accounts()['accounts']

In [ ]:
from exampleauth import exampleAuth
accountID, access_token = exampleAuth()

In [ ]:
try:
    instruments = "DE30_EUR,EUR_USD,EUR_JPY"
    r = PricingStream(accountID=accountID, params={"instruments":instruments})    
    print r
    print r.METHOD
    print r.status_code
    print r.response
    #print client.request(r)
    rv = client.request(r)    
    print rv
    #for ri in rv:
    #    ri
except Exception as e:
    print e

In [ ]:
try:
    r = AccountSummary(accountID="101-004-1984564-001")
    print r
    print r.METHOD
    print r.status_code
    print r.response
    rv = client.request(r)
    print p.DataFrame(rv)
except Exception as e:
    print e

In [ ]:
try:
    r = trades.TradesList(accountID=accid)
    print r
    print r.METHOD
    print r.status_code
    print r.response
    rv = client.request(r)
    mmdf = p.DataFrame(rv['trades'])
    p.to_numeric(mmdf['initialUnits'])
    mmdf['initialUnits'] = p.to_numeric(mmdf['initialUnits'])
    mmdf['side'] = map(lambda x: 'buy' if x > 0 else 'sell', mmdf['initialUnits'])
    mmdf['units'] = n.absolute(mmdf['initialUnits'])
    print mmdf
except Exception as e:
    print e

In [ ]:
try:
    r = accounts.AccountDetails(accountID)
    print r
    print r.METHOD
    print r.status_code
    print r.response
    rv = client.request(r)
    print p.DataFrame(rv)
except Exception as e:
    print e

In [ ]:
print client.environment

In [ ]:
from oandapyV20.exceptions import V20Error
import oandapyV20.endpoints.pricing as pricing
# list of requests
lor = []
# request trades list
res = trades.TradesList(accountID)
print res
print res.ENDPOINT
lor.append(res)
# request accounts list
lor.append(accounts.AccountList())
# request pricing info
params={"instruments": "DE30_EUR,EUR_GBP"}
lor.append(pricing.PricingInfo(accountID, params=params))

for r in lor:
    try:
        rv = client.request(r)
        # put request and response in 1 JSON structure
        print("{}".format(json.dumps({"request": "{}".format(r),
                                      "response": rv}, indent=2)))
    except V20Error as e:
        print("OOPS: {:d} {:s}".format(e.code, e.msg))

In [ ]:
import json
import pandas as p
from oandapyV20 import API    # the client
import oandapyV20.endpoints.trades as trades
import oandapyV20.endpoints.accounts as accounts

access_token = "f55f8626416a19d784732a4a19ec69d0-2828876dabe1996cfedfa96f50e6d36b"
accountID = "101-004-1984564-002"
client = API(access_token=access_token)

al = accounts.AccountList()
ral = client.request(al)
ral
df = p.DataFrame(ral['accounts'])
print df

print
print df.ix[0,'id']

# request trades list
try:
    r = trades.TradesList(accountID)
    #print r
    rv = client.request(r)
    print("RESPONSE:\n{}".format(json.dumps(rv, indent=2)))
except Exception as e:
    print e

In [ ]:
print accounts.AccountInstruments

In [ ]:
print instruments.Instruments

In [ ]:
print instruments.APIRequest

In [ ]:
print instruments.Instruments

In [ ]:
print instruments.InstrumentsCandles

In [ ]:
import v20
api = v20.Context(hostname='api-fxpractice.oanda.com', port=443, ssl=True, token='e6ab562b039325f12a026c6fdb7b71bb-b3d8721445817159410f01514acd19hbc')
#streaming_hostname: stream-fxpracatice.oanda.com
#username: user
#accounts:
#- 101-001-100000-001
#- 101-001-100000-002
#active_account: 101-001-100000-001
api.primitives.

In [ ]:
import numpy as n
print p.DataFrame(n.power(2, n.array(range(7+1))))

In [ ]:
#source: https://www.youtube.com/watch?v=DIIUNtUVDPI
import pandas as p
import numpy as n
#import matplotlib.pylab as plt

%pylab inline

df = p.DataFrame([1,2,3,4])
df[1] = n.power(df[0],2)
df[2] = n.power(df[0],3)
df[3] = n.power(df[0],n.e)
print df
df.plot()
#plt.show()

In [ ]:
%reload_ext autoreload
%autoreload 2
import sys
def defp(pt):
    try:    sys.path.index(pt)
    except: sys.path.append(pt)
defp('/ml.dev/bin')
defp('/ml.dev/lib/oanda/oandapy')
from qoreliquid import getSyntheticCurrencyTable
from oandaq import OandaQ
import pandas as p
import numpy as n
#oq = OandaQ(selectOandaAccount=0, selectOandaSubAccount=6)
#df = getSyntheticCurrencyTable(oq.oanda2, oq, ['EUR_GBP','AUD_USD','NZD_USD'])#.transpose()
#print df.ix[:,'pairedCurrencyAsk pairedCurrencyBid'.split(' ')]
#print df.ix[:,'pairedCurrencyAsk pairedCurrencyBid'.split(' ')]
#print df

In [ ]:
import sys
def defp(pt):
    try:    sys.path.index(pt)
    except: sys.path.append(pt)
defp('/mldev/lib/bitcoin/poloniex/thiagof_py-poloniex.github.py.git')
%pylab inline
from poloniex import Poloniex
from datetime import datetime
from datetime import timedelta
from qoreliquid import normalizeme
from qoreliquid import sigmoidme
pol = Poloniex('M8YTJIKE-2EIE8VV2-7UP6Z9O0-PJNGRPV4', '7ed5f13cee6469c3790f236c28d9b9dcd3b1714f9b5a310c8f25f221348ec5c2d0149cda35d58f53a34cae20d8e246f8462d7e95abcaa2bac4062ad24fb0034d')

In [ ]:
#pol.returnChartData()
start = datetime.now() - timedelta(hours=24)
df = {}
#bss = ['BTC_ETH', 'BTC_DOGE']
bss = list(p.DataFrame(pol.returnBalances().keys()).sort_values(by=0).transpose().get_values()[0])
bss = map(lambda x: 'BTC_%s'%x, bss)
#print bss
mdf=p.DataFrame([])
for i in bss[0:6]:
    print '--- %s' % i
    try:
        res = pol.returnChartData(i, 300, start)
        df[i] = p.DataFrame(res).sort_values(by='date').set_index('date')
        #with p.option_context('display.max_rows', 4000, 'display.max_columns', 4000, 'display.width', 1000000): print df[i].tail(15)
        mdf[i] = df[i].ix[:,'close']
    except Exception as e:
        #print e
        #print res['error']
        ''
#df.ix[:,'close'].plot()
mdf = mdf.ffill().bfill()
mdf = normalizeme(mdf)
mdf = sigmoidme(mdf)
print mdf
mdf.plot()

In [ ]:
mdf#.ix[:,0:6]#.plot(legend=0)`\

In [ ]:
#start = datetime.now() - timedelta(hours=1)
#res = pol.returnTradeHistory('BTC_DOGE', start)
#df=p.DataFrame(res)
#df=df.convert_objects(convert_numeric=True)
#print df.dtypes
gdf=df.groupby('type')
gdfsum=gdf.sum()
#gdfsum gdfsum.transpose()1
gdfsum.ix['total', 'total'] = - gdfsum.ix['buy','total'] + gdfsum.ix['sell','total']
print gdfsum
print gdfsum.ix['buy','total']
print gdfsum.ix['sell','total']


In [ ]:
import sys
def defp(pt):
    try:    sys.path.index(pt)
    except: sys.path.append(pt)
defp('/ml.dev/bin')
defp('/ml.dev/lib/oanda/oandapy')
from qoreliquid import normalizeme
from qoreliquid import sigmoidme
import pandas as p
import oandapy
from pylab import rcParams
%pylab inline
rcParams['figure.figsize'] = 20, 5

co = p.read_csv('/mldev/bin/datafeeds/config.csv', header=None)

co.ix[1,4] = 'oanda1'
env1=co.ix[1,1]
access_token1=co.ix[1,2]
oanda1 = oandapy.API(environment=env1, access_token=access_token1)

co.ix[0,4] = 'oanda2'
env2=co.ix[0,1]
access_token2=co.ix[0,2]
oanda2 = oandapy.API(environment=env2, access_token=access_token2)

try:
    acc = oanda2.get_accounts()['accounts']
    #print 'acc'
    #print p.DataFrame(acc)
    accid = acc[2]['accountId']
    #print 'using account: {0}'.format(accid)
except:
    ''

In [ ]:
co

In [ ]:
import pymongo as mong
import ujson
mongo = mong.MongoClient(host='127.0.0.1', port=3311)
dfm = p.DataFrame()
for i in mongo.ql.broker_oanda_accounts.find():
    #print p.DataFrame(i, index=[0]).transpose()
    #print i['user']
    #print i['utctime']
    df = p.DataFrame(ujson.loads(i['data']))
    df['user']    = i['user']
    df['utctime'] = i['utctime']
    for j in df.index:
        df.ix[j, 'idx'] = '%s-%s-%s' % (df.ix[j, 'user'], df.ix[j, 'utctime'], j)
    df['indx'] = df.index
    df = df.set_index('idx')
    #print df
    dfm = dfm.combine_first(df)
dfm = dfm.sort_values(by=['user', 'accountId'], ascending=[True, True])#.tail(20)
###
#aa = 'balance marginAvail marginUsed realizedPl unrealizedPl openTrades'
aa = 'balance marginUsed realizedPl unrealizedPl openTrades'
bb = 'utctime %s' % aa
ffds = aa.split(' ')
#print dfm.where(dfm['accountName'] == 'sub1')
#print dfm
#print dfm[dfm['accountName'] == 'sub1']
df = dfm[dfm['accountId'] == 947325].sort_values(by=['utctime'], ascending=[True]).ix[:, bb.split(' ')]
df = normalizeme(df)
df = sigmoidme(df)
#print df
%pylab inline
#rcParams['figure.figsize'] = 20, 5
plot(df.ix[:, ffds])
legend(ffds, loc=2)
#show()

In [ ]:
#from matplotlib.pylab import plot, legend, title, show #, imshow, tight_layout
#from pylab import rcParams
%pylab inline
#rcParams['figure.figsize'] = 20, 5
plot(df)
show()

In [ ]:
# circle of wealth
cc = [0.5, 1, 200.0/15, 2000.0/15, 2500.0/15, 50, 500, 1000, 2400, 5000]
from qorequant import QoreQuant
qq = QoreQuant(oandaInit=False, statWingInit=False)
print qq.calcCapitalRequirements(cc)
#print calcCapital(5000)